<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup-&amp;-Installation" data-toc-modified-id="Setup-&amp;-Installation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup &amp; Installation</a></span></li><li><span><a href="#GPU-Hackathon" data-toc-modified-id="GPU-Hackathon-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>GPU Hackathon</a></span><ul class="toc-item"><li><span><a href="#Reflected-Light-Functions" data-toc-modified-id="Reflected-Light-Functions-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Reflected Light Functions</a></span></li><li><span><a href="#Thermal-Emission-Functions" data-toc-modified-id="Thermal-Emission-Functions-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Thermal Emission Functions</a></span></li><li><span><a href="#Transmission--Functions" data-toc-modified-id="Transmission--Functions-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Transmission  Functions</a></span></li></ul></li><li><span><a href="#Initial-Setup-of-PICASO" data-toc-modified-id="Initial-Setup-of-PICASO-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Initial Setup of <code>PICASO</code></a></span><ul class="toc-item"><li><span><a href="#Reflected-Light-Run" data-toc-modified-id="Reflected-Light-Run-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Reflected Light Run</a></span></li><li><span><a href="#Thermal-Emission-Run" data-toc-modified-id="Thermal-Emission-Run-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Thermal Emission Run</a></span></li><li><span><a href="#Transmission-Run" data-toc-modified-id="Transmission-Run-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Transmission Run</a></span></li></ul></li><li><span><a href="#GPU-Test-Functions-Directly" data-toc-modified-id="GPU-Test-Functions-Directly-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>GPU Test Functions Directly</a></span></li></ul></div>

# Setup & Installation

1. Fork: https://github.com/natashabatalha/picaso
2. `git checkout gpu_hack`
3. `python setup.py develop`
4. [Download reference data & setup environment variables](https://natashabatalha.github.io/picaso/installation.html#download-reference-documentation)
5. This notebook can be downloaded here

# GPU Hackathon 

There are three functions that we want to enable GPU capability for the three different kinds of planet observations: reflected light, thermal emission, and transmission spectroscopy.

![](https://natashabatalha.github.io/picaso/_images/usecases.png)

## Reflected Light Functions

Function to focus on: [get_reflected_1d](https://github.com/natashabatalha/picaso/blob/fb2d48a2f2201ba104b3cd7235a7709764b2582a/picaso/fluxes.py#L827)

Functions that it calls: 

- [setup_tri_diag](https://github.com/natashabatalha/picaso/blob/fb2d48a2f2201ba104b3cd7235a7709764b2582a/picaso/fluxes.py#L240)

This sets up matries for a tridiagonal matrix problem 

- [solve_tri_diag](https://github.com/natashabatalha/picaso/blob/fb2d48a2f2201ba104b3cd7235a7709764b2582a/picaso/fluxes.py#L445) 

This is the tridiagonal matrix solver 


## Thermal Emission Functions

Function to focus on: [get_thermal_1d](https://github.com/natashabatalha/picaso/blob/fb2d48a2f2201ba104b3cd7235a7709764b2582a/picaso/fluxes.py#L1357)

Functions that it calls (same as reflected_1d). 

## Transmission  Functions

Function to focus on: [get_transit_1d](https://github.com/natashabatalha/picaso/blob/fb2d48a2f2201ba104b3cd7235a7709764b2582a/picaso/fluxes.py#L1793)

No function calls


# Initial Setup of `PICASO`


More reading on these functions can be [found here](https://natashabatalha.github.io/picaso/notebooks/1_GetStarted.html). 

In [1]:
#picaso
from picaso import justdoit as jdi 
from picaso import justplotit as jpi
import numpy as np

jpi.output_notebook()

Loading BokehJS ...

The speed of the calculation will depend on how big this opacity bundle is. Specifically: 

1. Number of wavelength points in the calculation
2. Number of pressure layers 

In [2]:
opacity = jdi.opannection(wave_range=[0.3,1]) #We can use wave_range to control the number of wave points

In [3]:
opacity.nwno

18242

In [4]:
start_case = jdi.inputs()
#phase angle 
start_case.phase_angle(0) #radians

#define gravity
start_case.gravity(radius=1, radius_unit=jdi.u.Unit('R_jup'),
                   mass=1, mass_unit=jdi.u.Unit('M_jup'),
                  ) #any astropy units available
start_case.inputs['surface_reflect'] = 0 
start_case.inputs['hard_surface'] = 0 
#define star 
start_case.star(opacity, 5000,0,4.0) #opacity db, pysynphot database, temp, metallicity, logg 

In [5]:
#set atmospheric chemistry with this example case
start_case.atmosphere(filename=jdi.jupiter_pt(), delim_whitespace=True)

Sets layers which defines next dimension

In [6]:
#61 pressure levels 
start_case.inputs['atmosphere']['profile'].shape[0]

61

## Reflected Light Run 

In [7]:
df = start_case.spectrum(opacity,calculation='reflected')

## Thermal Emission Run 

In [8]:
df = start_case.spectrum(opacity,calculation='thermal')

## Transmission Run 

In [9]:
df = start_case.spectrum(opacity,calculation='transmission')

# GPU Test Functions Directly

The function above does a lot of initial setup. However, the focus of this hackathon should only be the three functions. So in `gpu_hack` I have created a dummy function called `gpu_test` that produces the output for the three functions that we want to test. 



In [10]:
reflected_input, thermal_input, transmission_input = jdi.gpu_test(start_case, opacity
)

### get_thermal_1d (priority # 1)

In [12]:
out = jdi.get_thermal_1d(*thermal_input)

### get_transit_1d (priority # 2)

In [13]:
out = jdi.get_transit_1d(*transmission_input)

### get_reflected_1d (priority # 3) 


In [11]:
out = jdi.get_reflected_1d(*reflected_input)